### Import

In [2]:
# 데이터 처리 
import pandas as pd
import numpy as np
# 시각화
import matplotlib.pyplot as plt
import seaborn as sns
import folium
# 파일 관리 
import os

## EDA

### Read Data

`-` 데이터 확인하기.

In [5]:
os.getcwd()

'C:\\Users\\oldcar\\Documents\\GitHub\\2022-AnalysisCompetition-Subway'

In [6]:
for path, _, files in os.walk(os.getcwd() + '\\Data'):
    for filename in files:
        print(os.path.join(path, filename))

C:\Users\oldcar\Documents\GitHub\2022-AnalysisCompetition-Subway\Data\Found Data\공기질_승객수_통합.csv
C:\Users\oldcar\Documents\GitHub\2022-AnalysisCompetition-Subway\Data\Found Data\서울교통공사_역사건축정보_20200331.csv
C:\Users\oldcar\Documents\GitHub\2022-AnalysisCompetition-Subway\Data\Found Data\서울교통공사_역사심도정보_20200929.csv
C:\Users\oldcar\Documents\GitHub\2022-AnalysisCompetition-Subway\Data\Found Data\서울교통공사_월별승하차인원_20201231.csv
C:\Users\oldcar\Documents\GitHub\2022-AnalysisCompetition-Subway\Data\Found Data\서울교통공사_환기구정보_202006.csv
C:\Users\oldcar\Documents\GitHub\2022-AnalysisCompetition-Subway\Data\Found Data\좌표.csv
C:\Users\oldcar\Documents\GitHub\2022-AnalysisCompetition-Subway\Data\Given Data\aircondition_2020.xlsx
C:\Users\oldcar\Documents\GitHub\2022-AnalysisCompetition-Subway\Data\Given Data\data1.csv
C:\Users\oldcar\Documents\GitHub\2022-AnalysisCompetition-Subway\Data\Given Data\data2.csv
C:\Users\oldcar\Documents\GitHub\2022-AnalysisCompetition-Subway\Data\Given Data\data3.csv
C:\Users\

`-` 데이터 가져오기. 

In [16]:
congestion=pd.read_csv('Data/Given Data/data1.csv') # 지하철 혼잡도 데이터
aircondition = pd.read_csv('Data/Given Data/data2.csv') # 지하철 공기질 데이터 
station = pd.read_csv('Data/Given Data/data3.csv') # 지하철 정보 데이터
passengers = pd.read_csv('Data/Given Data/passengers2020.csv') # 지하철 별 승객 수 데이터

### Preprocessing

#### congestion

In [32]:
congestion.head() #전처리 필요없음. 
print(congestion.isnull().sum()) #널값이 있지만 해당 시각에 운행하지 않은 정보도 필요
print(congestion.shape)

조사일자           0
호선             0
역번호            0
역명             0
구분             0
5:30          32
6:00          10
6:30          10
7:00          10
7:30          10
8:00          10
8:30          10
9:00          10
9:30          10
10:00         10
10:30         10
11:00         10
11:30         10
12:00         10
12:30         10
13:00         10
13:30         10
14:00         10
14:30         10
15:00         10
15:30         10
16:00         10
16:30         10
17:00         10
17:30         10
18:00         10
18:30         10
19:00         10
19:30         10
20:00         10
20:30         10
21:00         10
21:30         10
22:00         10
22:30         10
23:00         10
23:30         22
24:00:00    1112
24:30:00    1112
dtype: int64
(1668, 44)


#### aircondition

In [17]:
aircondition.head()

,호선,역사명,미세먼지(PM10),이산화탄소(CO2),포름알데히드(HCHO),일산화탄소(CO)
0,1,서울역1,76.4,627,16.4,0.6
1,1,시청1,100.7,610,17.8,1.0
2,1,종각,99.8,687,20.5,0.9
3,1,종로3가1,105.3,646,20.6,1.3
4,1,종로5가,99.4,532,16.4,1.1


공기질 점수를 만들어야 하니 가중합을 사용해야한다. 

In [37]:
line_num = [f'{i}' for i in range(1,9)] # 8호선까지만 있는것 확인
line_num

[1, 2, 3, 4, 5, 6, 7, 8]

In [23]:
aircondition['호선']

SyntaxError: invalid syntax (4193547217.py, line 1)